In [1]:
#niezbędne biblioteki
import gym
import random
import numpy as np
from keras.models     import Sequential
from keras.layers     import Dense, Conv2D, Flatten
from keras.optimizers import Adam
from keras.utils import np_utils
from skimage import color

Using TensorFlow backend.


In [45]:
env = gym.make('Enduro-v0')
env.reset()
goal_steps = 4800
#score_requirement = 6000
intial_games = 1000

In [4]:
#do zobaczenia informacji jakie zwraca nam gra
def play_a_random_game_first():
    for step_index in range(goal_steps):
        #env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print("Step {}:".format(step_index))
        print("action: {}".format(action))
        print("observation: {}".format(observation))
        print("reward: {}".format(reward))
        print("done: {}".format(done))
        print("info: {}".format(info))
        if done:
            break
    env.reset()

In [3]:
#funkcja do preprocesingu ramek. Ucinam górną część (niesitotne niebo) i zmieniam na czarno-biały obrazek
def preprocces_frame(frame):
    img = color.rgb2gray(frame)
    img=img[51:,:]
    img=img.reshape(img.shape[0], img.shape[1],-1)
    return img

In [31]:
#generowanie danych. Zapisuje tylko ścieżki, które dały dobry rezultat
def model_data_preparation(require_score):
    training_data = []
    observations=[]
    actions=[]
    i=0
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 8)
            observation, reward, done, info = env.step(action)
            observation = preprocces_frame(observation)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            score += 1
            if done:
                break
        if score >= require_score:
            for data in game_memory:
                data[1]=np_utils.to_categorical(data[1],8).reshape(8)
                observations.append(data[0])
                actions.append(data[1])

        env.reset()
        print(i)
        i+=1
    
    return [observations, actions]


In [ ]:
training_data = model_data_preparation(4500)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [43]:
training_data

[[], []]

In [17]:
#Funkcja do budowania modelu: prawdopodobnie będę generował i uczył model jednocześnie, by oszczędzić pamięć
def build_model():
    model = Sequential()
    model.add(Conv2D(30, (3,3), input_shape=(159,160, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(8, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam())

    return model


In [18]:
def train_model(training_data):
    X = np.array(training_data[0])
    y = np.array(training_data[1])
    model = build_model()
    
    model.fit(X, y, epochs=10)
    return model

In [19]:
trained_model = train_model(training_data)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
4995/4995 [==============================] - 38s 8ms/step - loss: 2.7668
Epoch 2/10
4995/4995 [==============================] - 32s 6ms/step - loss: 2.0747
Epoch 3/10
4995/4995 [==============================] - 32s 6ms/step - loss: 2.0622
Epoch 4/10
4995/4995 [==============================] - 32s 6ms/step - loss: 2.0415
Epoch 5/10
4995/4995 [==============================] - 32s 6ms/step - loss: 2.0088
Epoch 6/10
4995/4995 [==============================] - 32s 6ms/step - loss: 1.9684
Epoch 7/10
4995/4995 [==============================] - 32s 6ms/step - loss: 1.9076
Epoch 8/10
4995/4995 [==============================] - 32s 6ms/step - loss: 1.8340
Epoch 9/10
4995/4995 [==============================] - 32s 6ms/step - loss: 1.7624
Epoch 10/10
4995/4995 [==============================] - 32s 6ms/step - loss: 1.6615


In [29]:
#Testowanie
scores = []
#choices = []
for each_game in range(20):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        #env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, prev_obs.shape[0], prev_obs.shape[1],
                                                                      prev_obs.shape[2])))
        
        #choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = preprocces_frame(new_observation)
        score+=1
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:', sum(scores)/len(scores))
#print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))

[4443, 4431, 4456, 4436, 4411, 4444, 4442, 4448, 4430, 4438, 4435, 4440, 4461, 4435, 4391, 4440, 4429, 4412, 4450, 4429]
Average Score: 4435.05


In [30]:
#Dla porównania losowe ruchy
scores = []
#choices = []
for each_game in range(20):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
        #env.render()
        action = random.randrange(0,8)
        
        #choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        score+=1
        if done:
            break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:', sum(scores)/len(scores))

[4432, 4387, 4467, 4450, 4463, 4422, 4417, 4462, 4418, 4414, 4466, 4414, 4437, 4444, 4409, 4441, 4443, 4473, 4454, 4448]
Average Score: 4438.05


In [6]:
from skimage import color
new_observation, reward, done, info = env.step(1)
img = color.rgb2gray(new_observation)

In [11]:
img=img.reshape(210, 160, -1)

In [12]:
img.shape

(210, 160, 1)